# gettingFacetsInPDF

- This is a notebook to create interfacial facets in vector format. 
- It takes the output from `./getFacet2D` and plots the facets in a PDF.
- It also calculates rMax and vMax using the output from `./getRmaxNV`. You can also look at the logFile.dat that is created while running the simulation to get rMax(t) which will have a much better time frequency as compared to the output from `./getRmaxNV` (which is restricted to number of snapshots we same instead of saving at every time step as in logFile.dat).

In [1]:
import numpy as np
import os
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.ticker import StrMethodFormatter

## Function Descriptions

### Style Setup
- Creates a custom color map using white to brown gradient colors
- Configures matplotlib to use serif fonts and LaTeX for text rendering


#### `gettingFacets(filename, includeCoat='true')`
- Executes `getFacet2D` to extract interface coordinates
- Processes the output to create line segments for plotting
- Creates symmetric segments by mirroring across r and z axes
- Returns list of line segments representing the interface

#### `gettingfield(filename, zmin, zmax, rmax, nr)`
- Executes `getData-elastic-scalar2D` to get field data
- Extracts and processes 5 key variables:
  - R, Z: Coordinate matrices
  - D2: Velocity gradient tensor norm
  - vel: Velocity field
  - taup: Elastic energy (normalized by G)
- Reshapes data into 2D arrays for plotting
- Returns processed field data and grid dimensions

#### `process_timestep(t, folder, GridsPerR, rmin, rmax, zmin, zmax, lw)`
- Main plotting function that processes a single timestep
- Takes direct time value `t` instead of calculating from index
- Creates visualization with:
  - Interface boundaries (blue/green lines)
  - Domain boundaries and axes
  - Two colormaps:
    - Left: Velocity gradient tensor norm (hot_r colormap)
    - Right: Elastic energy (custom brown colormap)
- Saves output as PDF with proper axes, labels, and colorbars
- Designed to work with multiprocessing for parallel execution

In [6]:
import matplotlib.colors as mcolors
custom_colors = ["white", "#DA8A67", "#A0522D", "#400000"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("custom_hot", custom_colors)

matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

def gettingFacets(filename,includeCoat='true'):
    exe = ["./getFacet2D", filename, includeCoat]
    p = sp.Popen(exe, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    temp1 = stderr.decode("utf-8")
    temp2 = temp1.split("\n")
    segs = []
    skip = False
    if (len(temp2) > 1e2):
        for n1 in range(len(temp2)):
            temp3 = temp2[n1].split(" ")
            if temp3 == ['']:
                skip = False
                pass
            else:
                if not skip:
                    temp4 = temp2[n1+1].split(" ")
                    r1, z1 = np.array([float(temp3[1]), float(temp3[0])])
                    r2, z2 = np.array([float(temp4[1]), float(temp4[0])])
                    segs.append(((r1, z1),(r2, z2)))
                    segs.append(((-r1, z1),(-r2, z2)))
                    segs.append(((r1, -z1),(r2, -z2)))
                    segs.append(((-r1, -z1),(-r2, -z2)))
                    skip = True
    return segs

def gettingfield(filename, zmin, zmax, rmax, nr):
    exe = ["./getData-elastic-scalar2D", filename, str(0.), str(0.), str(zmax), str(rmax), str(nr)]
    p = sp.Popen(exe, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    temp1 = stderr.decode("utf-8")
    temp2 = temp1.split("\n")
    # print(temp2) #debugging
    Rtemp, Ztemp, D2temp, veltemp, taupTemp  = [],[],[],[],[]

    for n1 in range(len(temp2)):
        temp3 = temp2[n1].split(" ")
        if temp3 == ['']:
            pass
        else:
            Ztemp.append(float(temp3[0]))
            Rtemp.append(float(temp3[1]))
            D2temp.append(float(temp3[2]))
            veltemp.append(float(temp3[3]))
            taupTemp.append(float(temp3[4]))

    R = np.asarray(Rtemp)
    Z = np.asarray(Ztemp)
    D2 = np.asarray(D2temp)
    vel = np.asarray(veltemp)
    taup = np.asarray(taupTemp)
    nz = int(len(Z)/nr)

    # print("nr is %d %d" % (nr, len(R))) # debugging
    print("nz is %d" % nz)

    R.resize((nz, nr))
    Z.resize((nz, nr))
    D2.resize((nz, nr))
    vel.resize((nz, nr))
    taup.resize((nz, nr))

    return R, Z, D2, vel, taup, nz
# ----------------------------------------------------------------------------------------------------------------------

def process_timestep(t, folder, GridsPerR, rmin, rmax, zmin, zmax, lw):
    place = f"intermediate/snapshot-{t:.4f}"
    name = f"{folder}/{int(t*1000):08d}.pdf"

    if not os.path.exists(place):
        print(f"{place} File not found!")
        return

    if os.path.exists(name):
        print(f"{name} Image present!")
        return

    segs1 = gettingFacets(place)
    segs2 = gettingFacets(place, 'false')

    if not segs1 and not segs2:
        print(f"Problem in the available file {place}")
        return

    nr = int(GridsPerR * rmax)
    R, Z, taus, vel, taup, nz = gettingfield(place, zmin, zmax, rmax, nr)
    zminp, zmaxp, rminp, rmaxp = Z.min(), Z.max(), R.min(), R.max()

    # Plotting
    AxesLabel, TickLabel = 50, 20
    fig, ax = plt.subplots()
    fig.set_size_inches(19.20, 10.80)

    ax.plot([0, 0], [zmin, zmax], '-.', color='grey', linewidth=lw)
    ax.plot([rmin, rmax], [0., 0.], '--', color='grey', linewidth=lw)
    ax.plot([rmin, rmin], [zmin, zmax], '-', color='black', linewidth=lw)
    ax.plot([rmin, rmax], [zmin, zmin], '-', color='black', linewidth=lw)
    ax.plot([rmin, rmax], [zmax, zmax], '-', color='black', linewidth=lw)
    ax.plot([rmax, rmax], [zmin, zmax], '-', color='black', linewidth=lw)

    line_segments = LineCollection(segs2, linewidths=4, colors='green', linestyle='solid')
    ax.add_collection(line_segments)
    line_segments = LineCollection(segs1, linewidths=4, colors='blue', linestyle='solid')
    ax.add_collection(line_segments)

    cntrl1 = ax.imshow(taus, cmap="hot_r", interpolation='Bilinear', origin='lower', extent=[-rminp, -rmaxp, zminp, zmaxp], vmax=3.0, vmin=-3.0)
    cntrl1 = ax.imshow(taus, cmap="hot_r", interpolation='Bilinear', origin='lower', extent=[-rminp, -rmaxp, -zminp, -zmaxp], vmax=3.0, vmin=-3.0)

    # TODO: fixme the colorbar bounds for taup must be set manually based on the simulated case.
    cntrl2 = ax.imshow(taup, interpolation='Bilinear', cmap=custom_cmap, origin='lower', extent=[rminp, rmaxp, zminp, zmaxp], vmax=3.0, vmin=-3.0)
    cntrl2 = ax.imshow(taup, interpolation='Bilinear', cmap=custom_cmap, origin='lower', extent=[rminp, rmaxp, -zminp, -zmaxp], vmax=3.0, vmin=-3.0)

    ax.set_aspect('equal')
    ax.set_xlim(rmin, rmax)
    ax.set_ylim(zmin, zmax)
    ax.set_title(f'$t\\Omega_c$ = {t:4.3f}', fontsize=TickLabel)

    l, b, w, h = ax.get_position().bounds
    # Left colorbar
    cb1 = fig.add_axes([l-0.04, b, 0.03, h])
    c1 = plt.colorbar(cntrl1, cax=cb1, orientation='vertical')
    c1.set_label(r'$\log_{10}\left(\|\mathcal{D}\|\right)$', fontsize=TickLabel, labelpad=5)
    c1.ax.tick_params(labelsize=TickLabel)
    c1.ax.yaxis.set_ticks_position('left')
    c1.ax.yaxis.set_label_position('left')
    c1.ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.1f}'))
    
    # Right colorbar
    cb2 = fig.add_axes([l+w+0.01, b, 0.03, h])
    c2 = plt.colorbar(cntrl2, cax=cb2, orientation='vertical')
    c2.ax.tick_params(labelsize=TickLabel)
    c2.set_label(r'$\log_{10}\left(\text{tr}\left(\mathcal{A}\right)-1\right)$', fontsize=TickLabel)
    c2.ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
    ax.axis('off')

    plt.savefig(name, bbox_inches="tight")
    plt.close()

This is actual code run. Please note the difference between this run and [VideoAxi.py](VideoAxi.py). 

In [7]:
ZMAX = 1.05
RMAX = 4.0
ZMIN = -1.05
    
rmin, rmax, zmin, zmax = [-RMAX, RMAX, ZMIN, ZMAX]
GridsPerR = 100

lw = 2
folder = 'VideoPDF'
if not os.path.isdir(folder):
    os.makedirs(folder)

t = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for Time in t:
    process_timestep(Time, folder, GridsPerR, rmin, rmax, zmin, zmax, lw)
    print("done:", Time, "\n")

nz is 105
done: 0.0 

nz is 105
done: 0.1 

nz is 105
done: 0.2 

nz is 105
done: 0.3 

nz is 105
done: 0.4 

nz is 105
done: 0.5 

nz is 105
done: 0.6 

nz is 105
done: 0.7 

nz is 105
done: 0.8 

nz is 105
done: 0.9 

nz is 105
done: 1.0 

